In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


In [2]:
left_volume_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Volumetric_Analysis\lh_aparc_volume_20231117.csv"
left_volume = pd.read_csv(left_volume_file_path)
left_volume = pd.DataFrame(left_volume)

right_volume_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Volumetric_Analysis\rh_aparc_volume_20231117.csv"
right_volume = pd.read_csv(right_volume_file_path)
right_volume = pd.DataFrame(right_volume)


In [3]:
left_volume.head()
right_volume.head()

,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,rh_rostralanteriorcingulate_volu,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume
0,1001,1,2,1,1,5,16.0,7.0,4335.4,2167.7,...,1672.0,15816.0,20720.0,15523.0,14333.0,10071.0,1184.0,1685.0,1095.0,8970.0
1,1002,1,2,1,1,5,15.0,14.0,10363.1,5708.1,...,2593.0,18136.0,26808.0,17638.0,16253.0,14687.0,1316.0,1339.0,1036.0,8656.0
2,1003,1,2,1,1,5,18.0,12.0,6685.4,4863.9,...,1713.0,14229.0,21140.0,14419.0,11672.0,9538.0,1102.0,2295.0,877.0,6745.0
3,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,1702.0,13092.0,20406.0,14824.0,10843.0,12316.0,958.0,1746.0,1107.0,5813.0
4,1005,1,3,0,2,5,21.0,NaN,NaN,NaN,...,2483.0,15323.0,17971.0,13579.0,12156.0,8855.0,1135.0,2205.0,922.0,7298.0


In [4]:
print("Column Names:")
print(right_volume.columns[2])

Column Names:
checkin_bin


In [6]:
#group them base on the value in the third column which indicates their level of playing
right_grouped = right_volume.groupby(right_volume.iloc[:, 2])
left_grouped = left_volume.groupby(left_volume.iloc[:, 2])

NFL_right_grouped = pd.DataFrame()
CP_right_grouped = pd.DataFrame()
HC_right_grouped = pd.DataFrame()


# group_name : 1, 2, 3   group_data: 
for group_name, group_data in right_grouped:
    if group_name == 1:
        NFL_right_grouped = pd.concat([NFL_right_grouped,group_data], ignore_index = True)
    if group_name == 2:
        CP_right_grouped = pd.concat([CP_right_grouped,group_data], ignore_index = True)
    if group_name == 3:
        HC_right_grouped = pd.concat([HC_right_grouped,group_data], ignore_index = True)
    
#print("DataFrame for NFL:")
#print(NFL_right_grouped.head())
NFL_right_grouped.head()


,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,rh_rostralanteriorcingulate_volu,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume
0,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,1702.0,13092.0,20406.0,14824.0,10843.0,12316.0,958.0,1746.0,1107.0,5813.0
1,1008,1,1,1,2,3,15.0,22.0,8220.9,5421.2,...,1810.0,14049.0,18025.0,12089.0,9251.0,7607.0,843.0,2381.0,763.0,6106.0
2,1011,1,1,1,2,5,16.0,20.0,9307.0,9307.0,...,2336.0,17565.0,22340.0,15661.0,10571.0,11334.0,1103.0,1795.0,1022.0,6678.0
3,1015,1,1,1,1,3,19.0,17.0,9866.7,6173.3,...,1683.0,14775.0,18342.0,12055.0,11520.0,8808.0,1082.0,1790.0,1060.0,6808.0
4,1018,1,1,1,1,3,16.0,23.0,10635.9,7929.6,...,1724.0,16225.0,20546.0,11500.0,11241.0,11318.0,939.0,2681.0,812.0,7453.0


In [7]:
index_of_Atlas = NFL_right_grouped.columns.get_loc("Atlas")
NFL_right_grouped.drop(columns=NFL_right_grouped.columns[[1] +list(range(3,index_of_Atlas+1))], inplace = True)
CP_right_grouped.drop(columns=CP_right_grouped.columns[[1] +list(range(3,index_of_Atlas+1))], inplace = True)
HC_right_grouped.drop(columns=HC_right_grouped.columns[[1] +list(range(3,index_of_Atlas+1))], inplace = True)

NFL_right_grouped.head()

,subject_id,checkin_bin,rh_bankssts_volume,rh_caudalanteriorcingulate_volum,rh_caudalmiddlefrontal_volume,rh_cuneus_volume,rh_entorhinal_volume,rh_fusiform_volume,rh_inferiorparietal_volume,rh_inferiortemporal_volume,...,rh_rostralanteriorcingulate_volu,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume
0,1004,1,2310.0,1647.0,4656.0,3471.0,1326.0,7496.0,13607.0,8159.0,...,1702.0,13092.0,20406.0,14824.0,10843.0,12316.0,958.0,1746.0,1107.0,5813.0
1,1008,1,1946.0,1687.0,4961.0,3116.0,1317.0,6869.0,12892.0,8378.0,...,1810.0,14049.0,18025.0,12089.0,9251.0,7607.0,843.0,2381.0,763.0,6106.0
2,1011,1,1961.0,2483.0,6019.0,4356.0,677.0,7605.0,17582.0,7795.0,...,2336.0,17565.0,22340.0,15661.0,10571.0,11334.0,1103.0,1795.0,1022.0,6678.0
3,1015,1,2092.0,2032.0,5290.0,3845.0,1500.0,7774.0,13624.0,10242.0,...,1683.0,14775.0,18342.0,12055.0,11520.0,8808.0,1082.0,1790.0,1060.0,6808.0
4,1018,1,2547.0,2028.0,5994.0,4281.0,1406.0,9650.0,14401.0,10641.0,...,1724.0,16225.0,20546.0,11500.0,11241.0,11318.0,939.0,2681.0,812.0,7453.0


In [8]:
#combine all three classes
combined_right_volume = pd.concat([NFL_right_grouped, CP_right_grouped, HC_right_grouped], ignore_index=True)

combined_right_volume.head()

,subject_id,checkin_bin,rh_bankssts_volume,rh_caudalanteriorcingulate_volum,rh_caudalmiddlefrontal_volume,rh_cuneus_volume,rh_entorhinal_volume,rh_fusiform_volume,rh_inferiorparietal_volume,rh_inferiortemporal_volume,...,rh_rostralanteriorcingulate_volu,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume
0,1004,1,2310.0,1647.0,4656.0,3471.0,1326.0,7496.0,13607.0,8159.0,...,1702.0,13092.0,20406.0,14824.0,10843.0,12316.0,958.0,1746.0,1107.0,5813.0
1,1008,1,1946.0,1687.0,4961.0,3116.0,1317.0,6869.0,12892.0,8378.0,...,1810.0,14049.0,18025.0,12089.0,9251.0,7607.0,843.0,2381.0,763.0,6106.0
2,1011,1,1961.0,2483.0,6019.0,4356.0,677.0,7605.0,17582.0,7795.0,...,2336.0,17565.0,22340.0,15661.0,10571.0,11334.0,1103.0,1795.0,1022.0,6678.0
3,1015,1,2092.0,2032.0,5290.0,3845.0,1500.0,7774.0,13624.0,10242.0,...,1683.0,14775.0,18342.0,12055.0,11520.0,8808.0,1082.0,1790.0,1060.0,6808.0
4,1018,1,2547.0,2028.0,5994.0,4281.0,1406.0,9650.0,14401.0,10641.0,...,1724.0,16225.0,20546.0,11500.0,11241.0,11318.0,939.0,2681.0,812.0,7453.0


In [9]:
# Separate based on the level of professionalism
X = combined_right_volume.drop(columns='checkin_bin')  # Adjust 'Label' to the actual column name containing class labels
y = combined_right_volume['checkin_bin']

In [10]:
# Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

In [11]:
# Normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# Replace NaNs with means
imputer = SimpleImputer(strategy='mean')  # You can choose a different strategy
X_train_imputed = imputer.fit_transform(X_train_scaled)
X_test_imputed = imputer.transform(X_test_scaled)

In [13]:
# train the model
model = RandomForestClassifier()
model.fit(X_train_imputed, y_train)

RandomForestClassifier()

In [14]:
# prediction
y_pred = model.predict(X_test_imputed)

In [15]:
# Evaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

Accuracy: 0.5120481927710844, Precision: 0.48701007990237444, Recall: 0.5120481927710844, F1-Score: 0.49035758455048184
